In [1]:
#Research references:
#1) Dry/wet cough classification: https://link.springer.com/article/10.1007/s10439-013-0741-6
#2) Pneumonia classification: https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=6987276
#3) https://espace.library.uq.edu.au/data/UQ_344963/s41943203_phd_submission.pdf?Expires=1585601065&Key-Pair-Id=APKAJKNBJ4MJBJNC6NLQ&Signature=Lnpf6wT8rkozSh9av7U9nGuC7WAH6KuI2Cj3Y7G366gkGlh8D-Ie1Kc~TyBAUu~uMsVltleJcSv3p6TCm6HdFnhpyoTgLcYh6eFfvQwIUqbk1Bf4JZldgB~BDKUOwY1G0pA-HoKjvIAu3avO98SMO35upakm9OEBByd4nC9aXsjKRThd6bTpq1qIuuD9gh1l5FaM6hNRB0c2lCf4Q3adx7C3FW0NMwdWhcuF45A9f~dO3zTWWSQamoo5Otc-PHMMt96TetNcML~jy9ghgJeCPY6DJLUIwQAt03fENBluS~TjTJ17WD~n51xiRofb94fEJHoRHh0d-430LLwr7BX4IA__

In [2]:
import numpy as np
import os
#import pywt #wavelets
from pydub import AudioSegment
from pydub.silence import split_on_silence
from pydub.utils import mediainfo
from pydub.playback import play
import matplotlib.pyplot as plt
#import seaborn as sn
import python_speech_features as spe_feats
import pandas as pd
from scipy.stats import kurtosis, skew
from scipy.signal import lfilter
import librosa
import pysptk
import math
import sys
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold
#from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score, confusion_matrix

#settings
import config

## Reading recordings

In [3]:
all_s=[]
all_label=[]
all_id=[]
all_fs=[]

In [4]:
#Read wav data set

if config.featExtr_skip is False:
    print("Readings wavs...")

    #only list files in FOLDER_PATH directory
    wav_files = [f for f in os.listdir(config.FOLDER_PATH) if os.path.isfile(os.path.join(config.FOLDER_PATH, f))]
    for file_name in wav_files:
    
        fname_noExt = os.path.splitext(file_name)[0] #file name without extension
    
        #full path file name
        full_fname = config.FOLDER_PATH+file_name
        print(full_fname)
    
        # load audio
        s = AudioSegment.from_wav(full_fname)
        print(full_fname)
        all_s.append(s)
        #sampling rate:
        info = mediainfo(full_fname)
        fs = float(info['sample_rate'])
        all_fs.append(fs)
    
        #get ID of recording
        ID = fname_noExt.split('-')[-2] #for the current type of naming
        #print(file_name)
        #print(ID)
        all_id.append(ID)
    
        #get label
        label = fname_noExt.split('-')[-1] #for the current type of naming
        #print(label)
        all_label.append(label)

Readings wavs...
data/YT_set/edited_wavs/edit_Spring Allergy Coughing-7Ez5Wc_esBg-Dry.wav
data/YT_set/edited_wavs/edit_Spring Allergy Coughing-7Ez5Wc_esBg-Dry.wav
data/YT_set/edited_wavs/edit_Coughing 51-LkxvBb2VXbs-Dry.wav
data/YT_set/edited_wavs/edit_Coughing 51-LkxvBb2VXbs-Dry.wav
data/YT_set/edited_wavs/edit_Wet coughing-0QQxKN-KC1U-Wet.wav
data/YT_set/edited_wavs/edit_Wet coughing-0QQxKN-KC1U-Wet.wav
data/YT_set/edited_wavs/edit_Dry Coughing Fit in the Afternoon.-A5s2ZgwQ1VM-Dry.wav
data/YT_set/edited_wavs/edit_Dry Coughing Fit in the Afternoon.-A5s2ZgwQ1VM-Dry.wav
data/YT_set/edited_wavs/edit_Coughing 77-2Mw-s5jnqXU-Wet.wav
data/YT_set/edited_wavs/edit_Coughing 77-2Mw-s5jnqXU-Wet.wav
data/YT_set/edited_wavs/edit_Wheezing Chest and Wet Cough 2-5905FxXz9dI-Wet.wav
data/YT_set/edited_wavs/edit_Wheezing Chest and Wet Cough 2-5905FxXz9dI-Wet.wav
data/YT_set/edited_wavs/edit_Coughing 46-dg-I9j76-t8-Wet.wav
data/YT_set/edited_wavs/edit_Coughing 46-dg-I9j76-t8-Wet.wav
data/YT_set/edited_

Listening to some of the audios

In [5]:
if config.featExtr_skip is False:
    np.where(np.array(all_label)=='Dry')

In [6]:
if config.featExtr_skip is False:
    np.where(np.array(all_label)=='Wet')

In [7]:
if config.featExtr_skip is False:
    s=all_s[15]
    s

## Feature extraction

In [8]:
import featureExtractionFunctions

if config.featExtr_skip is False:

    feats = featureExtractionFunctions.feature_extraction_Step(all_s,all_id,all_label)
    
       

High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass 

## Load  (or store) features 

In [9]:

feats_fname = 'feats_df.pkl'

if config.featExtr_skip is False:
    #Store feature df
    feats.to_pickle(feats_fname)
else:
    #Load feature df
    feats = pd.read_pickle(feats_fname)

## Pre-processing of features

In [10]:
feats

,Id,kurt,logEnergy,zcr,F0,skewness,label,entropy,mfcc_0,mfcc_1,...,mfcc_7,mfcc_8,mfcc_9,mfcc_10,mfcc_11,mfcc_12,F1,F2,F3,F4
0,7Ez5Wc_esBg,3.706714,2.510354,0.641604,457.142857,-0.350596,Dry,8.643856,12.154072,-53.889708,...,-13.203568,-3.480999,0.217583,-11.359071,-23.046762,9.816940,NaN,NaN,NaN,NaN
1,7Ez5Wc_esBg,7.762735,2.599776,0.761905,432.432432,-0.312852,Dry,8.643856,12.473961,-56.181480,...,1.236424,0.631101,-5.085036,6.876460,-13.753278,4.849485,NaN,NaN,NaN,NaN
2,7Ez5Wc_esBg,1.893479,2.469615,0.729323,444.444444,0.127164,Dry,8.643856,12.188062,-54.491861,...,-7.354098,-2.208687,-4.561046,-5.488364,-12.229855,-4.461634,NaN,NaN,NaN,NaN
3,7Ez5Wc_esBg,1.919692,2.738322,0.789474,516.129032,-0.054807,Dry,8.643856,12.892992,-51.279293,...,-12.276520,6.892395,-1.984168,7.735193,-18.072184,-4.649738,NaN,NaN,NaN,NaN
4,7Ez5Wc_esBg,2.692210,2.450934,0.791980,500.000000,-0.125955,Dry,8.643856,12.171122,-52.351841,...,-9.025626,-2.758992,-10.323113,-4.270525,-14.132265,3.084379,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4465,ct3tHDfNKiQ,1.596083,5.224528,0.626566,484.848485,-0.004247,Wet,8.643856,18.315415,-59.223086,...,2.850233,-6.934126,-5.118358,-13.225567,-15.709984,2.325254,NaN,NaN,NaN,NaN
4466,ct3tHDfNKiQ,1.698796,5.326481,0.644110,500.000000,-0.173511,Wet,8.643856,18.526675,-58.655791,...,-8.142383,-27.868522,-5.177583,-15.276977,-21.546500,11.333826,NaN,NaN,NaN,NaN
4467,ct3tHDfNKiQ,2.658432,5.181313,0.629073,457.142857,-0.000887,Wet,8.643856,18.197910,-61.197791,...,-10.318832,-16.866847,2.686336,-12.883587,-15.328687,9.527345,NaN,NaN,NaN,NaN
4468,ct3tHDfNKiQ,3.703883,5.159381,0.629073,457.142857,0.269426,Wet,8.643856,18.150308,-60.377438,...,-11.427861,-27.715255,-16.008879,-18.162084,-23.158347,14.375599,NaN,NaN,NaN,NaN


In [11]:
#1.Check which columns have NaNs values

#feats2 = feats.copy()

#sum(feats.isna().any())
#feats.columns[feats.isna().any()].tolist() --> We get just the ones we have inserted in formants
#feats2 = feats.interpolate(method ='cubic')
feats2 = feats.dropna(axis=1).copy()
#feats2.dropna(axis=0, how="any", thresh=None, subset=None, inplace=True)

#feats2.columns[feats2.isna().any()].tolist()
#feats2.describe()


In [12]:
sum(feats2.isna().any())

0

In [13]:
feats2

,Id,kurt,logEnergy,zcr,F0,skewness,label,entropy,mfcc_0,mfcc_1,...,mfcc_3,mfcc_4,mfcc_5,mfcc_6,mfcc_7,mfcc_8,mfcc_9,mfcc_10,mfcc_11,mfcc_12
0,7Ez5Wc_esBg,3.706714,2.510354,0.641604,457.142857,-0.350596,Dry,8.643856,12.154072,-53.889708,...,4.734463,-1.427698,2.646440,-4.617411,-13.203568,-3.480999,0.217583,-11.359071,-23.046762,9.816940
1,7Ez5Wc_esBg,7.762735,2.599776,0.761905,432.432432,-0.312852,Dry,8.643856,12.473961,-56.181480,...,0.919350,-4.894177,-10.937749,-2.942003,1.236424,0.631101,-5.085036,6.876460,-13.753278,4.849485
2,7Ez5Wc_esBg,1.893479,2.469615,0.729323,444.444444,0.127164,Dry,8.643856,12.188062,-54.491861,...,3.888765,4.601985,-5.545437,4.465844,-7.354098,-2.208687,-4.561046,-5.488364,-12.229855,-4.461634
3,7Ez5Wc_esBg,1.919692,2.738322,0.789474,516.129032,-0.054807,Dry,8.643856,12.892992,-51.279293,...,-12.123837,-4.883726,-13.128090,6.232161,-12.276520,6.892395,-1.984168,7.735193,-18.072184,-4.649738
4,7Ez5Wc_esBg,2.692210,2.450934,0.791980,500.000000,-0.125955,Dry,8.643856,12.171122,-52.351841,...,-5.048541,-7.284575,-17.192872,-5.794937,-9.025626,-2.758992,-10.323113,-4.270525,-14.132265,3.084379
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4465,ct3tHDfNKiQ,1.596083,5.224528,0.626566,484.848485,-0.004247,Wet,8.643856,18.315415,-59.223086,...,-4.565820,-3.300138,8.717628,6.850718,2.850233,-6.934126,-5.118358,-13.225567,-15.709984,2.325254
4466,ct3tHDfNKiQ,1.698796,5.326481,0.644110,500.000000,-0.173511,Wet,8.643856,18.526675,-58.655791,...,1.980749,5.344209,1.882938,0.138826,-8.142383,-27.868522,-5.177583,-15.276977,-21.546500,11.333826
4467,ct3tHDfNKiQ,2.658432,5.181313,0.629073,457.142857,-0.000887,Wet,8.643856,18.197910,-61.197791,...,-2.332589,2.217049,0.096397,-1.695375,-10.318832,-16.866847,2.686336,-12.883587,-15.328687,9.527345
4468,ct3tHDfNKiQ,3.703883,5.159381,0.629073,457.142857,0.269426,Wet,8.643856,18.150308,-60.377438,...,-5.241304,2.328262,0.690715,0.029509,-11.427861,-27.715255,-16.008879,-18.162084,-23.158347,14.375599


In [14]:
#Grouping the frames from a same recording (Id) into chunks with the same number of frames.
#The training of the classifier will be based on these chunks

feats2['cum_IDidx'] = feats2.groupby('Id').cumcount()

def get_subidx(cum_Idx,batch_size):
    #batch needs to be an integer (or float like 3.0)
    return int(1.0*cum_Idx/batch_size)

feats2['subIdx'] = feats2.apply(lambda x: get_subidx(x['cum_IDidx'], 10), axis=1)
feats2 = feats2.drop(['cum_IDidx'],axis=1)

In [15]:
feats2

,Id,kurt,logEnergy,zcr,F0,skewness,label,entropy,mfcc_0,mfcc_1,...,mfcc_4,mfcc_5,mfcc_6,mfcc_7,mfcc_8,mfcc_9,mfcc_10,mfcc_11,mfcc_12,subIdx
0,7Ez5Wc_esBg,3.706714,2.510354,0.641604,457.142857,-0.350596,Dry,8.643856,12.154072,-53.889708,...,-1.427698,2.646440,-4.617411,-13.203568,-3.480999,0.217583,-11.359071,-23.046762,9.816940,0
1,7Ez5Wc_esBg,7.762735,2.599776,0.761905,432.432432,-0.312852,Dry,8.643856,12.473961,-56.181480,...,-4.894177,-10.937749,-2.942003,1.236424,0.631101,-5.085036,6.876460,-13.753278,4.849485,0
2,7Ez5Wc_esBg,1.893479,2.469615,0.729323,444.444444,0.127164,Dry,8.643856,12.188062,-54.491861,...,4.601985,-5.545437,4.465844,-7.354098,-2.208687,-4.561046,-5.488364,-12.229855,-4.461634,0
3,7Ez5Wc_esBg,1.919692,2.738322,0.789474,516.129032,-0.054807,Dry,8.643856,12.892992,-51.279293,...,-4.883726,-13.128090,6.232161,-12.276520,6.892395,-1.984168,7.735193,-18.072184,-4.649738,0
4,7Ez5Wc_esBg,2.692210,2.450934,0.791980,500.000000,-0.125955,Dry,8.643856,12.171122,-52.351841,...,-7.284575,-17.192872,-5.794937,-9.025626,-2.758992,-10.323113,-4.270525,-14.132265,3.084379,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4465,ct3tHDfNKiQ,1.596083,5.224528,0.626566,484.848485,-0.004247,Wet,8.643856,18.315415,-59.223086,...,-3.300138,8.717628,6.850718,2.850233,-6.934126,-5.118358,-13.225567,-15.709984,2.325254,11
4466,ct3tHDfNKiQ,1.698796,5.326481,0.644110,500.000000,-0.173511,Wet,8.643856,18.526675,-58.655791,...,5.344209,1.882938,0.138826,-8.142383,-27.868522,-5.177583,-15.276977,-21.546500,11.333826,11
4467,ct3tHDfNKiQ,2.658432,5.181313,0.629073,457.142857,-0.000887,Wet,8.643856,18.197910,-61.197791,...,2.217049,0.096397,-1.695375,-10.318832,-16.866847,2.686336,-12.883587,-15.328687,9.527345,11
4468,ct3tHDfNKiQ,3.703883,5.159381,0.629073,457.142857,0.269426,Wet,8.643856,18.150308,-60.377438,...,2.328262,0.690715,0.029509,-11.427861,-27.715255,-16.008879,-18.162084,-23.158347,14.375599,11


In [16]:
mean_feats = feats2.groupby(['Id','subIdx']).aggregate('mean').reset_index()
std_feats = feats2.groupby(['Id','subIdx']).agg(lambda x: x.std(ddof=0)).reset_index() #ddof=0 to compute population std (rather than sample std)
keep_same = {'Id', 'subIdx'}
mean_feats.columns = ['{}{}'.format(c, '' if c in keep_same else '_m') for c in mean_feats.columns]
std_feats.columns = ['{}{}'.format(c, '' if c in keep_same else '_std') for c in std_feats.columns]

In [17]:
sum(std_feats.isna().any())

0

In [18]:
mean_std_feats = pd.merge(mean_feats, std_feats, on=['Id','subIdx'], how='outer')

In [19]:
#Make dictionary and add label column using it 
feats_unique = feats.drop_duplicates(subset=['Id'])
label_dict = dict(zip(feats_unique.Id, feats_unique.label))
mean_std_feats['label'] = mean_std_feats["Id"].map(label_dict)
#mean_std_feats[['Id','label']].head(50)

In [20]:
mean_std_feats.columns

Index(['Id', 'subIdx', 'kurt_m', 'logEnergy_m', 'zcr_m', 'F0_m', 'skewness_m',
       'entropy_m', 'mfcc_0_m', 'mfcc_1_m', 'mfcc_2_m', 'mfcc_3_m', 'mfcc_4_m',
       'mfcc_5_m', 'mfcc_6_m', 'mfcc_7_m', 'mfcc_8_m', 'mfcc_9_m', 'mfcc_10_m',
       'mfcc_11_m', 'mfcc_12_m', 'F0_std', 'entropy_std', 'kurt_std',
       'logEnergy_std', 'mfcc_0_std', 'mfcc_1_std', 'mfcc_10_std',
       'mfcc_11_std', 'mfcc_12_std', 'mfcc_2_std', 'mfcc_3_std', 'mfcc_4_std',
       'mfcc_5_std', 'mfcc_6_std', 'mfcc_7_std', 'mfcc_8_std', 'mfcc_9_std',
       'skewness_std', 'zcr_std', 'label'],
      dtype='object')

In [21]:
mean_std_feats.describe()

,subIdx,kurt_m,logEnergy_m,zcr_m,F0_m,skewness_m,entropy_m,mfcc_0_m,mfcc_1_m,mfcc_2_m,...,mfcc_2_std,mfcc_3_std,mfcc_4_std,mfcc_5_std,mfcc_6_std,mfcc_7_std,mfcc_8_std,mfcc_9_std,skewness_std,zcr_std
count,465.000000,465.000000,465.000000,465.000000,465.000000,465.000000,465.000000,465.000000,465.000000,465.000000,...,465.000000,465.000000,465.000000,465.000000,465.000000,465.000000,465.000000,465.000000,465.000000,465.000000
mean,6.460215,3.891690,4.243611,0.676807,440.164773,-0.003837,8.499361,16.115227,-46.109345,-3.914013,...,4.363459,6.011631,6.492005,6.846066,6.898470,7.955112,8.459117,7.990919,0.236467,0.041937
std,4.458021,3.410781,1.985224,0.164188,26.851309,0.220395,0.616400,4.677118,8.536427,8.423421,...,1.878562,2.660700,2.787865,2.670177,2.827098,3.284760,3.295593,3.245578,0.214387,0.034886
min,0.000000,0.605327,-0.921195,0.072682,337.732114,-1.783253,1.594310,4.137886,-61.395391,-27.441467,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,1.947150,2.666927,0.493734,424.025976,-0.049218,8.631363,12.296160,-52.903060,-9.658168,...,3.095411,4.019280,4.780379,5.065030,5.179107,5.692797,6.188364,5.892106,0.100754,0.023867
50%,6.000000,2.476054,5.014310,0.729574,442.252243,-0.005165,8.643856,17.793403,-47.318143,-4.219281,...,4.045961,5.417462,5.891193,6.377610,6.485465,7.430721,7.931842,7.554355,0.147863,0.033386
75%,10.000000,5.825867,5.791059,0.812030,457.805752,0.040007,8.643856,19.863885,-40.583802,1.390664,...,5.268821,7.583406,7.615271,8.018979,8.180107,9.771119,10.348892,9.594265,0.312380,0.047184
max,20.000000,35.066899,6.997471,0.945614,516.129032,3.600665,8.643856,22.621448,-6.754285,25.128835,...,13.459443,16.058676,20.071410,17.924307,20.027083,24.112487,21.593752,25.261439,2.012421,0.362797


In [22]:
mean_std_feats.columns[mean_std_feats.isna().any()].tolist() 

[]

In [23]:
mean_std_feats['Id']

0      1UDFq2InljM
1      1UDFq2InljM
2      1UDFq2InljM
3      1UDFq2InljM
4      1UDFq2InljM
          ...     
460    zjd4HrJbc8o
461    zjd4HrJbc8o
462    zjd4HrJbc8o
463    zjd4HrJbc8o
464    zjd4HrJbc8o
Name: Id, Length: 465, dtype: object

In [24]:
sum(mean_std_feats.isna().any())

0

In [25]:
#mean_std_feats = mean_std_feats.interpolate(method ='cubic')
#mean_std_feats.dropna(axis=0, how="any", thresh=None, subset=None, inplace=True) #--> Doesn't work!? Still get NaN error

In [26]:
sum(mean_std_feats.isna().any())

0

In [27]:
mean_std_feats['Id']

0      1UDFq2InljM
1      1UDFq2InljM
2      1UDFq2InljM
3      1UDFq2InljM
4      1UDFq2InljM
          ...     
460    zjd4HrJbc8o
461    zjd4HrJbc8o
462    zjd4HrJbc8o
463    zjd4HrJbc8o
464    zjd4HrJbc8o
Name: Id, Length: 465, dtype: object

In [28]:
#2. Get feature set, labels, and recording IDs
X_train = mean_std_feats.drop(['label','Id','subIdx'], 1).copy()
y_train =  mean_std_feats['label'].copy()

ID_train = mean_std_feats['Id']
ID_list = ID_train.drop_duplicates()

#ID_train.size
ID_list.size

36

In [29]:
#3. Normalization in case some model requires it

scaler = StandardScaler()
scaler.fit(X_train)

#use same scaler for both, based on X_train data
X_trainNorm = scaler.transform(X_train.values)

In [30]:
sum(X_train.isna().any())

0

## Model training

### Train-test split (k-fold)

In [31]:
k = ID_list.values.size #number of folds

group_kfold = GroupKFold(n_splits=k)
group_kfold.get_n_splits(X_trainNorm, y_train, ID_train)

36

In [32]:
k

36

### Logistic regression

In [33]:
#Do cross-validation
pred_probs = pd.DataFrame([])

idx_acc = 0
for train_index, test_index in group_kfold.split(X_trainNorm,y_train,ID_train):
    X_train1, X_test1 = X_trainNorm[train_index], X_trainNorm[test_index]
    y_train1, y_test1 = y_train[train_index], y_train[test_index]
    
    #TODO: optimize the penaly weight
    #https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegressionCV.html
    logReg = SGDClassifier(loss='log', penalty='elasticnet')
    logReg.fit(X_train1, y_train1)
    y_hat_prob = logReg.predict_proba(X_test1)
    classes =logReg.classes_
    pred_probs = pred_probs.append(pd.DataFrame({'ID': ID_train[test_index], str(classes[0]): y_hat_prob[:,0], str(classes[1]): y_hat_prob[:,1]}),ignore_index=True, sort=False)    

In [34]:
#mean_feats = feats2.groupby(['Id','subIdx']).aggregate('mean').reset_index()
mean_pred_probs = pred_probs.groupby('ID').aggregate('mean').reset_index()

In [35]:
mean_pred_probs

,ID,Dry,Wet
0,1UDFq2InljM,0.551800,0.448200
1,4k0ziD0j5BI,0.280300,0.719700
2,5905FxXz9dI,0.657922,0.342078
3,6LK6yHtIung,0.403494,0.596506
4,7Ez5Wc_esBg,0.805963,0.194037
5,A5s2ZgwQ1VM,0.868802,0.131198
6,AQOeIVbhFm4,0.771389,0.228611
7,CTSLdNxN1cc,0.102571,0.897429
8,CsDXlt7Ei1c,0.397526,0.602474
9,DYfjPnty2Ho,0.206068,0.793932


In [36]:
def predict_class(prob_dry,prob_wet):
    if prob_dry > prob_wet :
        return 'Dry'
    else:
        return 'Wet'
    
mean_pred_probs['pred_class'] = mean_pred_probs.apply(lambda x: predict_class(x['Dry'], x['Wet']), axis=1)

In [37]:
#add actual classes
mean_pred_probs['label'] = mean_pred_probs["ID"].map(label_dict)

In [38]:
mean_pred_probs

,ID,Dry,Wet,pred_class,label
0,1UDFq2InljM,0.551800,0.448200,Dry,Dry
1,4k0ziD0j5BI,0.280300,0.719700,Wet,Wet
2,5905FxXz9dI,0.657922,0.342078,Dry,Wet
3,6LK6yHtIung,0.403494,0.596506,Wet,Dry
4,7Ez5Wc_esBg,0.805963,0.194037,Dry,Dry
5,A5s2ZgwQ1VM,0.868802,0.131198,Dry,Dry
6,AQOeIVbhFm4,0.771389,0.228611,Dry,Dry
7,CTSLdNxN1cc,0.102571,0.897429,Wet,Wet
8,CsDXlt7Ei1c,0.397526,0.602474,Wet,Wet
9,DYfjPnty2Ho,0.206068,0.793932,Wet,Wet


In [39]:
mean_pred_probs[(mean_pred_probs.pred_class != mean_pred_probs.label)]

,ID,Dry,Wet,pred_class,label
2,5905FxXz9dI,0.657922,0.342078,Dry,Wet
3,6LK6yHtIung,0.403494,0.596506,Wet,Dry
10,Dc_aoUCqw2E,0.695300,0.304700,Dry,Wet
15,LkxvBb2VXbs,0.211596,0.788404,Wet,Dry
16,NaOVmYoIjbs,0.433625,0.566375,Wet,Dry
20,Xe68,0.122556,0.877444,Wet,Dry
22,ct3tHDfNKiQ,0.883931,0.116069,Dry,Wet
23,d2wkdrScerU,0.635311,0.364689,Dry,Wet
25,h2FLCKMcEX0,0.589174,0.410826,Dry,Wet
27,oCg,0.027714,0.972286,Wet,Dry


## Evaluation

In [40]:
#Accuracy
acc = accuracy_score(mean_pred_probs['label'], mean_pred_probs['pred_class'])
print(acc)

0.6111111111111112


In [41]:
#TODO: Check if following measures are computed OK

In [42]:
#Precision
prec = precision_score(mean_pred_probs['label'], mean_pred_probs['pred_class'],average="macro")
print(prec)

0.6


In [43]:
#F1-score
f1 = f1_score(mean_pred_probs['label'], mean_pred_probs['pred_class'],average="macro")
print(f1)

0.6


In [44]:
#recall
recall = recall_score(mean_pred_probs['label'], mean_pred_probs['pred_class'],average="macro")
print(recall)

0.6


In [45]:
#confusion matrix
conf_mat_df = pd.crosstab(mean_pred_probs['label'], mean_pred_probs['pred_class'], margins=True)

In [46]:
conf_mat_df

pred_class,Dry,Wet,All
label,,,
Dry,8,7,15
Wet,7,14,21
All,15,21,36


In [47]:
#Cough sound
#Breathing rate
#Breathing rhytm (consistence smoothness)
#Cough rate
#Panic level
#Hoarseness